Made by: *Vincent Kwan*

In [39]:
#The first row is always the Headers
#The last column is always the label

training_data = []
with open('drug200.csv', 'r') as f:
    training_data = f.read().splitlines()
training_data = [x.split(',') for x in training_data]


In [40]:
intl = []
for c in range(len(training_data[1])):
    try:
        i = float(training_data[1][c])
        #print(c)
        intl.append(c)
    except Exception as E:
        continue

In [41]:
for i in intl:
    for r in range(1,len(training_data[1:])):
        training_data[r][i] = float(training_data[r][i])

In [42]:
headers = training_data[0]

In [43]:
testing_data = training_data[150:200]

In [44]:
testing_data

[[22.0, 'M', 'LOW', 'HIGH', 8.151, 'drugC'],
 [49.0, 'M', 'HIGH', 'NORMAL', 8.7, 'drugA'],
 [68.0, 'M', 'HIGH', 'HIGH', 11.009, 'drugB'],
 [55.0, 'M', 'NORMAL', 'NORMAL', 7.261, 'drugX'],
 [72.0, 'F', 'LOW', 'NORMAL', 14.642, 'drugX'],
 [37.0, 'M', 'LOW', 'NORMAL', 16.724, 'drugY'],
 [49.0, 'M', 'LOW', 'HIGH', 10.537, 'drugC'],
 [31.0, 'M', 'HIGH', 'NORMAL', 11.227, 'drugA'],
 [53.0, 'M', 'LOW', 'HIGH', 22.963, 'drugY'],
 [59.0, 'F', 'LOW', 'HIGH', 10.444, 'drugC'],
 [34.0, 'F', 'LOW', 'NORMAL', 12.923, 'drugX'],
 [30.0, 'F', 'NORMAL', 'HIGH', 10.443, 'drugX'],
 [57.0, 'F', 'HIGH', 'NORMAL', 9.945, 'drugB'],
 [43.0, 'M', 'NORMAL', 'NORMAL', 12.859, 'drugX'],
 [21.0, 'F', 'HIGH', 'NORMAL', 28.632, 'drugY'],
 [16.0, 'M', 'HIGH', 'NORMAL', 19.007, 'drugY'],
 [38.0, 'M', 'LOW', 'HIGH', 18.295, 'drugY'],
 [58.0, 'F', 'LOW', 'HIGH', 26.645, 'drugY'],
 [57.0, 'F', 'NORMAL', 'HIGH', 14.216, 'drugX'],
 [51.0, 'F', 'LOW', 'NORMAL', 23.003, 'drugY'],
 [20.0, 'F', 'HIGH', 'HIGH', 11.262, 'drugA'],

In [45]:
training_data = training_data[:149]

In [46]:
def uniques(rows, col):
    #return unique values for columns - skip the headers
    return set(row[col] for row in rows[1:])

In [47]:
def countlabels(rows):
    #count the number of labels in a node
    dict_label = dict()
    for row in rows:
        if row[-1] not in dict_label:
            dict_label[row[-1]] = 0
        dict_label[row[-1]] += 1
    return dict_label

In [48]:
countlabels(training_data[1:])

{'drugY': 69, 'drugC': 10, 'drugX': 40, 'drugA': 16, 'drugB': 13}

In [49]:
def gini_impurity(rows):
    #calculate the gini impurity for a node
    counts = countlabels(rows)
    total = sum(counts.values())
    impurity = 1
    for label in counts:
        prob = counts[label]/total
        prob = prob ** 2
        impurity -= prob
    return impurity

In [50]:
def weighted_impurity(lrow, rrow):
    #calculate weighted impurity of child nodes
    ltotal = len(lrow)
    rtotal = len(rrow)
    total = ltotal + rtotal
    w_imp = ((ltotal/total) * gini_impurity(lrow)) + ((rtotal/total) * gini_impurity(rrow))
    return w_imp

In [51]:
#testing if the function works, no mixing:
test1 = [['Red', 'Color'], ['Yellow', 'Color']]
gini_impurity(test1)

0.0

In [52]:
#some mixing
test2 = [['Ravioli', 'Food'], ['Paracetamol', 'Medicine'], ['Pug', 'Dog']]
gini_impurity(test2)

0.6666666666666665

In [53]:
class Condition:
    ''' A condition which is used to divide values into child nodes.'''
    def __init__(self, col, val):
        self.value = val
        self.column = col
    
    def is_match(self, row):
        test_val = row[self.column]

        if isinstance(test_val, float) or isinstance(test_val, int):
            #if data is of numeric type, then check whether bigger than or equal to
            return test_val >= self.value
        else:
            #otherwise, if data is of character type, then check equality
            return test_val == self.value

    def __repr__(self):
        if isinstance(self.value, float) or isinstance(self.value, int):
            operator = ' >= '
        else:
            operator = ' == '
        
        return (str(headers[self.column]) + operator + str(self.value))

In [54]:
#testing the Condition class
C = Condition(1, 'Biden')
trow = ['President', 'Biden']
C.is_match(trow)

True

In [55]:
def partitioner(rows, condition):
    #Partition the dataset based upon the condition
    leftrows, rightrows = [], []
    for row in rows:
        if condition.is_match(row):
            rightrows.append(row)
        else:
            leftrows.append(row)
    return leftrows, rightrows

In [56]:
#testing 
lr, rr = partitioner(training_data, Condition(0, 23))
weighted_impurity(lr, rr)

0.6859060402684564

In [57]:
lr

[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K', 'Drug'],
 [22.0, 'F', 'NORMAL', 'HIGH', 8.607, 'drugX'],
 [16.0, 'F', 'HIGH', 'NORMAL', 15.516, 'drugY'],
 [18.0, 'F', 'NORMAL', 'NORMAL', 8.75, 'drugX'],
 [15.0, 'M', 'NORMAL', 'HIGH', 9.084, 'drugX'],
 [17.0, 'M', 'NORMAL', 'NORMAL', 10.832, 'drugX'],
 [18.0, 'F', 'HIGH', 'NORMAL', 24.276, 'drugY'],
 [19.0, 'F', 'HIGH', 'HIGH', 13.313, 'drugA'],
 [20.0, 'M', 'HIGH', 'NORMAL', 35.639, 'drugY'],
 [15.0, 'F', 'HIGH', 'NORMAL', 16.725, 'drugY'],
 [22.0, 'M', 'HIGH', 'NORMAL', 28.294, 'drugY'],
 [22.0, 'M', 'NORMAL', 'HIGH', 11.953, 'drugX'],
 [20.0, 'F', 'NORMAL', 'NORMAL', 9.281, 'drugX'],
 [15.0, 'M', 'HIGH', 'NORMAL', 17.206, 'drugY'],
 [19.0, 'F', 'HIGH', 'NORMAL', 25.969, 'drugY']]

In [58]:
def splitter(rows):
    '''Finds the best condition to split the dataset on.'''
    best_con = None
    bg = 0
    curr_gini = gini_impurity(rows)
    
    for c in range(len(headers) - 1): #ignore the last column since it contains label
        values = uniques(rows, c)

        for v in values:
            condition = Condition(c, v)
            lrows, rrows = partitioner(rows, condition)

            if len(lrows) == 0 or len(rrows) == 0:
                continue
            
            w_imp = weighted_impurity(lrows, rrows)

            info_gain = curr_gini - w_imp

            if info_gain > bg:
                best_con = condition
                bg = info_gain

    return bg, best_con

In [59]:
#testing info_gain and best condition
bi, bc = splitter(training_data)
print(bi, bc)

0.33123170127471724 Na_to_K >= 15.015


In [60]:
class Node():
    def __init__(self, rows):
        self.labels = countlabels(rows)
        self.dprob = dict()

        self.total = sum(self.labels.values())
        for label in self.labels.keys():
            #print(label)
            self.dprob[label] = self.labels[label]/self.total

In [61]:
class LeafNode(Node):
    def __init__(self, rows):
        super(LeafNode, self).__init__(rows)
    
    def state(self):
        return 'I am a leaf node.'

In [62]:
class InternalNode(Node):
    def __init__(self, lrows, rrows, cond):
        self.state = 'I am a branch'
        self.condition = cond
        self.lbranch = lrows
        self.rbranch = rrows
    
    def state(self):
        return 'I am an internal node.'

In [63]:
def treebuilder(rows):
    gain, condition = splitter(rows)

    if gain == 0:
        return LeafNode(rows)

    lrows, rrows = partitioner(rows, condition)

    lbranch = treebuilder(lrows)
    rbranch = treebuilder(rrows)

    return InternalNode(lbranch, rbranch, condition)

In [64]:
tree = treebuilder(training_data[1:])

In [65]:
def print_tree(node, spacing=""):

    if isinstance(node, LeafNode):
        print (spacing + "Predict", node.dprob)
        return


    print (spacing + str(node.condition))


    print (spacing + '--> True:')
    print_tree(node.rbranch, spacing + "  ")


    print (spacing + '--> False:')
    print_tree(node.lbranch, spacing + "  ")

In [66]:
def print_node(row, treenode):
    
    if isinstance(treenode, LeafNode):
        dprob = treenode.dprob
        total = sum(dprob.values())
        probs = {}
        for lbl in dprob.keys():
            probs[lbl] = dprob[lbl]/total
        return probs

    if treenode.condition.is_match(row):
        return print_node(row, treenode.rbranch)
    else:
        return print_node(row, treenode.lbranch)

In [67]:
print_tree(tree)

Na_to_K >= 15.015
--> True:
  Predict {'drugY': 1.0}
--> False:
  BP == HIGH
  --> True:
    Age >= 51.0
    --> True:
      Predict {'drugB': 1.0}
    --> False:
      Predict {'drugA': 1.0}
  --> False:
    BP == NORMAL
    --> True:
      Predict {'drugX': 1.0}
    --> False:
      Cholesterol == HIGH
      --> True:
        Predict {'drugC': 1.0}
      --> False:
        Predict {'drugX': 1.0}


In [68]:
for row in testing_data:
    print("Actual: {0}. Predicted: {1}".format(row[-1], print_node(row, tree)))

Actual: drugC. Predicted: {'drugC': 1.0}
Actual: drugA. Predicted: {'drugA': 1.0}
Actual: drugB. Predicted: {'drugB': 1.0}
Actual: drugX. Predicted: {'drugX': 1.0}
Actual: drugX. Predicted: {'drugX': 1.0}
Actual: drugY. Predicted: {'drugY': 1.0}
Actual: drugC. Predicted: {'drugC': 1.0}
Actual: drugA. Predicted: {'drugA': 1.0}
Actual: drugY. Predicted: {'drugY': 1.0}
Actual: drugC. Predicted: {'drugC': 1.0}
Actual: drugX. Predicted: {'drugX': 1.0}
Actual: drugX. Predicted: {'drugX': 1.0}
Actual: drugB. Predicted: {'drugB': 1.0}
Actual: drugX. Predicted: {'drugX': 1.0}
Actual: drugY. Predicted: {'drugY': 1.0}
Actual: drugY. Predicted: {'drugY': 1.0}
Actual: drugY. Predicted: {'drugY': 1.0}
Actual: drugY. Predicted: {'drugY': 1.0}
Actual: drugX. Predicted: {'drugX': 1.0}
Actual: drugY. Predicted: {'drugY': 1.0}
Actual: drugA. Predicted: {'drugA': 1.0}
Actual: drugX. Predicted: {'drugX': 1.0}
Actual: drugX. Predicted: {'drugX': 1.0}
Actual: drugY. Predicted: {'drugY': 1.0}
Actual: drugY. P